# 01-OpenAI的API调用

! pip install openai langchain-openai

In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key, base_url=base_url, temperature=0.3)

llm.invoke("中国的首都是哪里?")


AIMessage(content='中国的首都是北京。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 16, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-55394ca9-4025-4f21-b093-1cc75709f3d1-0', usage_metadata={'input_tokens': 16, 'output_tokens': 7, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 基本示例: 提示+模型+输出解析器

最基本和常见的用例是将提示模板和模型链接在一起。为了看看这是如何工作的，让我们创建一个接受主题并生成小红书短文的链：

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("请帮我生成一篇关于{topic}的小红书短文")
model = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key, base_url=base_url, temperature=0.3)

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "胖东来烤鸭"})


'标题：胖东来烤鸭，让味蕾沉醉的美味\n\n在繁华的城市中，有一家让人回味无穷的烤鸭店——胖东来烤鸭。这家店虽然不起眼，但却是许多美食爱好者心中的美味天堂。\n\n走进胖东来烤鸭，首先映入眼帘的是烤鸭的诱人香气，让人垂涎欲滴。烤鸭皮金黄酥脆，肉质鲜嫩多汁，每一口都让人感受到满满的幸福。配上店家特制的鸭饼和葱丝，简直是绝配，让人欲罢不能。\n\n除了烤鸭，胖东来烤鸭还有许多其他美味的小吃，比如鸭血粉丝、鸭肠火锅等，每一道菜都让人惊艳。店内环境干净整洁，服务员热情周到，让人感受到家的温暖。\n\n每次来到胖东来烤鸭，都能让人的味蕾沉醉在美味之中，让人流连忘返。如果你也是美食爱好者，一定要来胖东来烤鸭品尝一番，相信你会爱上这里的美味，爱上这里的服务，爱上这里的一切。让我们一起来享受美食带来的幸福吧！'

## Prompt提示

prompt 是一个 BasePromptTemplate ，这意味着它接受模板变量的字典并生成一个 PromptValue 。 PromptValue 是一个完整提示的包装器，可以传递给 LLM （它接受一个字符串作为输入）或 ChatModel （它接受一个序列作为输入的消息）。它可以与任何一种语言模型类型一起使用，因为它定义了生成 BaseMessage 和生成字符串的逻辑。

In [3]:
prompt = ChatPromptTemplate.from_template("请根据下面的主题，帮我生成一篇小红书营销短文: {topic}")
prompt_value = prompt.invoke({"topic": "康师傅绿茶"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='请根据下面的主题，帮我生成一篇小红书营销短文: 康师傅绿茶', additional_kwargs={}, response_metadata={})])

## Model模型

Prompt_value被传给LLM，在本例中我们的LLM是ChatModel，意味着它将输BaseMessage

In [5]:
message = model.invoke(prompt_value)
message

AIMessage(content='标题：发现健康生活的新方式，康师傅绿茶带你一起走进清爽世界\n\n大家好，我是小红书的小编，今天我要和大家分享的是康师傅绿茶，一款清爽健康的饮品，让你在忙碌的生活中也能享受一丝清爽。\n\n康师傅绿茶选用优质绿茶叶精心制作而成，口感清爽，回味无穷。每一口都能感受到绿茶的清香和滋味，让人仿佛置身于清爽的绿色世界中。而且康师傅绿茶不含添加剂，健康又安心。\n\n在这个快节奏的生活中，我们常常忽略了身体的健康。康师傅绿茶提倡健康生活方式，让你在享受美味的同时也能注重身体健康。无论是工作中的疲惫还是生活中的压力，一瓶康师傅绿茶就能让你感受到清爽和舒适。\n\n康师傅绿茶，不仅是一款饮品，更是一种生活方式。让我们一起走进清爽的世界，发现健康生活的新方式。让康师傅绿茶成为你生活中的一部分，带给你清爽和健康。\n\n快来试试康师傅绿茶，让清爽成为你生活的一部分吧！愿我们每个人都能享受健康生活，活出自己的精彩！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 520, 'prompt_tokens': 47, 'total_tokens': 567, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-39555d5b-d44f-46b4-a20d-327d6e45988b-0', usage_metadata={'input_tokens': 47, 'output_tokens': 520, 'total_tokens'

## Output Parser输出解析器

将 model 输出传递给 output_parser ，这是一个 BaseOutputParser ，意味着它接受字符串或 BaseMessage 作为输入。 StrOutputParser 特别简单地将任何输入转换为字符串。

input ->(dict) -> PromptTemplate -> (prompt_value) -> ChatModel -> (ChatMessage) -> OutputParser -> (str) -> result

In [6]:
response = StrOutputParser().invoke(message)
response


'标题：发现健康生活的新方式，康师傅绿茶带你一起走进清爽世界\n\n大家好，我是小红书的小编，今天我要和大家分享的是康师傅绿茶，一款清爽健康的饮品，让你在忙碌的生活中也能享受一丝清爽。\n\n康师傅绿茶选用优质绿茶叶精心制作而成，口感清爽，回味无穷。每一口都能感受到绿茶的清香和滋味，让人仿佛置身于清爽的绿色世界中。而且康师傅绿茶不含添加剂，健康又安心。\n\n在这个快节奏的生活中，我们常常忽略了身体的健康。康师傅绿茶提倡健康生活方式，让你在享受美味的同时也能注重身体健康。无论是工作中的疲惫还是生活中的压力，一瓶康师傅绿茶就能让你感受到清爽和舒适。\n\n康师傅绿茶，不仅是一款饮品，更是一种生活方式。让我们一起走进清爽的世界，发现健康生活的新方式。让康师傅绿茶成为你生活中的一部分，带给你清爽和健康。\n\n快来试试康师傅绿茶，让清爽成为你生活的一部分吧！愿我们每个人都能享受健康生活，活出自己的精彩！'

## 流式输出

如果我们想流式传输结果，我们需要更改我们的函数，使其返回一个生成器。通过for循环打印输出效果

In [7]:

chain = prompt | model | StrOutputParser()

for chunk in chain.stream({"topic": "康师傅绿茶"}):
    print(chunk, end="", flush=True)


标题：探寻康师傅绿茶的魅力

大家好，今天我想和大家分享一下我对康师傅绿茶的独特看法。作为一款备受欢迎的饮品，康师傅绿茶不仅口感清爽，而且还有着让人难以抗拒的魅力。

首先，康师傅绿茶选用的茶叶品质上乘，口感清新自然，让人一口接一口欲罢不能。而且，康师傅绿茶采用的是独特的制茶工艺，保留了茶叶的原汁原味，让人在品尝的过程中仿佛置身于茶园之中。

其次，康师傅绿茶不仅口感上佳，而且还有着丰富的营养成分。茶叶中含有丰富的维生素和矿物质，对身体健康有着极大的益处。每一口康师傅绿茶都是对身体的呵护，让人在享受美味的同时也能获得健康。

最后，康师傅绿茶的包装设计简约大方，绿色的包装让人一眼就能联想到清新的茶香。无论是携带出门还是放在家中，康师傅绿茶都是一款让人爱不释手的饮品。

总的来说，康师傅绿茶不仅是一款口感清新的饮品，更是一种生活方式的选择。无论是在工作中还是在生活中，一杯康师傅绿茶都能让人感受到清新自然的美好。让我们一起来探寻康师傅绿茶的魅力，享受生活中的每一刻。